In [13]:
import numpy as np
import matplotlib.pyplot as plt
from galpy.orbit import Orbit
import astropy.units as u
from galpy.potential import MWPotential2014, vcirc
from galpy.potential.McMillan17 import McMillan17

In [14]:
def sample_exponential_disk(N, lR, lz, Rmax=None):
    """Sample stars from a double-exponential disk."""
    R = np.random.gamma(shape=2.0, scale=lR.value, size=N) * lR.unit
    z = np.random.choice([-1, 1], size=N) * np.random.exponential(lz.value, size=N) * lz.unit
    phi = np.random.uniform(0, 2*np.pi, size=N) * u.rad

    if Rmax is not None:
        mask = R < Rmax
        R, z, phi = R[mask], z[mask], phi[mask]

    x = R * np.cos(phi)
    y = R * np.sin(phi)
    return x, y, z


def sample_spherical_component(N, lR, Rmax=None):
    """Sample stars from a spherical exponential component (ρ ∝ e^{-R/lR})."""
    R = np.random.gamma(shape=3.0, scale=lR.value, size=N) * lR.unit
    if Rmax is not None:
        R = R[R < Rmax]

    phi = np.random.uniform(0, 2*np.pi, len(R))
    cos_theta = np.random.uniform(-1, 1, len(R))
    sin_theta = np.sqrt(1 - cos_theta**2)

    x = R * sin_theta * np.cos(phi)
    y = R * sin_theta * np.sin(phi)
    z = R * cos_theta
    return x, y, z

In [20]:
params = {
    "thin":  {"lR": 3.5*u.kpc, "lz": 0.3*u.kpc, "N": 1000, "color": "green"},
    "thick": {"lR": 2.9*u.kpc, "lz": 1.0*u.kpc, "N": 1000, "color": "lightblue"},
    "bulge": {"lR": 1.8*u.kpc, "N": 1000, "color": "red"},
    "halo":  {"lR": 15.0*u.kpc,"N": 1000, "color": "yellow"}
}

alpha_halo = 0.1
alpha_thic = 0.2
alpha_thin = 0.2
alpha_bulge = 0.05

Rmax_thin  = 50 * u.kpc
Rmax_thick = 50 * u.kpc
Rmax_bulge = 50 * u.kpc
Rmax_halo  = 50 * u.kpc


#  Generate all components
x_thin,  y_thin,  z_thin  = sample_exponential_disk(params["thin"]["N"],  params["thin"]["lR"],  params["thin"]["lz"],  Rmax=Rmax_thin)
x_thick, y_thick, z_thick = sample_exponential_disk(params["thick"]["N"], params["thick"]["lR"], params["thick"]["lz"], Rmax=Rmax_thick)
x_bulge, y_bulge, z_bulge = sample_spherical_component(params["bulge"]["N"], params["bulge"]["lR"], Rmax=Rmax_bulge)
x_halo,  y_halo,  z_halo  = sample_spherical_component(params["halo"]["N"],  params["halo"]["lR"],  Rmax=Rmax_halo)

In [21]:
# --- Common setup ---
ro = 8.0 * u.kpc
vo = 220.0 * u.km/u.s
t_end = 2 * u.Gyr
n_steps = 200
ts = np.linspace(0, t_end.value, n_steps) * u.Gyr

# --- Load McMillan potential ---
pot = McMillan17
for p in pot:
    if hasattr(p, "turn_physical_on"):
        p.turn_physical_on(ro=8.0, vo=220.0)

pot = MWPotential2014

print(f"✅  Potential ready for orbit integration.")

✅  Potential ready for orbit integration.


# Thin

In [17]:
# --- Parameters for thin disk ---
sigma_R, sigma_T, sigma_z = 30, 20, 15  # km/s dispersions
mean_vT = 220                           # mean azimuthal speed

# --- Initial conditions ---
x0, y0, z0 = x_thin.value, y_thin.value, z_thin.value
R0 = np.sqrt(x0**2 + y0**2)
phi0 = np.arctan2(y0, x0)
N = len(x0)

vR0 = np.random.normal(0, sigma_R, N) * u.km/u.s
vT0 = np.random.normal(mean_vT, sigma_T, N) * u.km/u.s
vz0 = np.random.normal(0, sigma_z, N) * u.km/u.s

# --- Create orbits ---
orbits_thin = [
    Orbit([R0[i]*u.kpc, vR0[i], vT0[i], z0[i]*u.kpc, vz0[i], phi0[i]*u.rad],
          ro=ro, vo=vo)
    for i in range(N)
]

# --- Integrate orbits ---
for o in orbits_thin:
    o.integrate(ts, pot, method='leapfrog')

print(f"✅ Integrated {N} thin disk orbits for {t_end}.")

# Helper functions to handle both unitful and unitless outputs
def safe_val(val, scale=1.0):
    """Return numeric value from a Galpy output (Quantity or float)."""
    if hasattr(val, "to"):
        return val.value
    else:
        return np.array(val) * scale

# Use ro (kpc) and vo (km/s) for scaling
ro_val = ro.value if hasattr(ro, "value") else ro
vo_val = vo.value if hasattr(vo, "value") else vo

# Extract initial and final data
x_i = np.array([safe_val(o.x(ts[0]), ro_val) for o in orbits_thin])
y_i = np.array([safe_val(o.y(ts[0]), ro_val) for o in orbits_thin])
z_i = np.array([safe_val(o.z(ts[0]), ro_val) for o in orbits_thin])

vx_i = np.array([safe_val(o.vx(ts[0]), vo_val) for o in orbits_thin])
vy_i = np.array([safe_val(o.vy(ts[0]), vo_val) for o in orbits_thin])
vz_i = np.array([safe_val(o.vz(ts[0]), vo_val) for o in orbits_thin])

x_f = np.array([safe_val(o.x(ts[-1]), ro_val) for o in orbits_thin])
y_f = np.array([safe_val(o.y(ts[-1]), ro_val) for o in orbits_thin])
z_f = np.array([safe_val(o.z(ts[-1]), ro_val) for o in orbits_thin])

vx_f = np.array([safe_val(o.vx(ts[-1]), vo_val) for o in orbits_thin])
vy_f = np.array([safe_val(o.vy(ts[-1]), vo_val) for o in orbits_thin])
vz_f = np.array([safe_val(o.vz(ts[-1]), vo_val) for o in orbits_thin])

# Save to file
data = np.column_stack([x_i, y_i, z_i, vx_i, vy_i, vz_i, x_f, y_f, z_f, vx_f, vy_f, vz_f])
header = "x_i[kpc] y_i[kpc] z_i[kpc] vx_i[km/s] vy_i[km/s] vz_i[km/s] x_f[kpc] y_f[kpc] z_f[kpc] vx_f[km/s] vy_f[km/s] vz_f[km/s]"
np.savetxt("orbits_thin.txt", data, header=header, fmt="%.6f")

print(f"✅ Saved {N} thin disk orbits to orbits_thin.txt")

✅ Integrated 999 thin disk orbits for 2.0 Gyr.
✅ Saved 999 thin disk orbits to orbits_thin.txt


# Thick

In [22]:
# --- Parameters for thick disk ---
sigma_R, sigma_T, sigma_z = 50, 40, 35  # km/s dispersions
mean_vT = 180                           # mean azimuthal speed

# --- Initial conditions ---
x0, y0, z0 = x_thick.value, y_thick.value, z_thick.value
R0 = np.sqrt(x0**2 + y0**2)
phi0 = np.arctan2(y0, x0)
N = len(x0)

vR0 = np.random.normal(0, sigma_R, N) * u.km/u.s
vT0 = np.random.normal(mean_vT, sigma_T, N) * u.km/u.s
vz0 = np.random.normal(0, sigma_z, N) * u.km/u.s

# --- Create orbits ---
orbits_thick = [
    Orbit([R0[i]*u.kpc, vR0[i], vT0[i], z0[i]*u.kpc, vz0[i], phi0[i]*u.rad],
          ro=ro, vo=vo)
    for i in range(N)
]

# --- Integrate orbits ---
for o in orbits_thick:
    o.integrate(ts, pot, method='leapfrog')

# Helper functions to handle both unitful and unitless outputs
def safe_val(val, scale=1.0):
    """Return numeric value from a Galpy output (Quantity or float)."""
    if hasattr(val, "to"):
        return val.value
    else:
        return np.array(val) * scale

# Use ro (kpc) and vo (km/s) for scaling
ro_val = ro.value if hasattr(ro, "value") else ro
vo_val = vo.value if hasattr(vo, "value") else vo

x_i = np.array([safe_val(o.x(ts[0]), ro_val) for o in orbits_thick])
y_i = np.array([safe_val(o.y(ts[0]), ro_val) for o in orbits_thick])
z_i = np.array([safe_val(o.z(ts[0]), ro_val) for o in orbits_thick])

vx_i = np.array([safe_val(o.vx(ts[0]), vo_val) for o in orbits_thick])
vy_i = np.array([safe_val(o.vy(ts[0]), vo_val) for o in orbits_thick])
vz_i = np.array([safe_val(o.vz(ts[0]), vo_val) for o in orbits_thick])

x_f = np.array([safe_val(o.x(ts[-1]), ro_val) for o in orbits_thick])
y_f = np.array([safe_val(o.y(ts[-1]), ro_val) for o in orbits_thick])
z_f = np.array([safe_val(o.z(ts[-1]), ro_val) for o in orbits_thick])

vx_f = np.array([safe_val(o.vx(ts[-1]), vo_val) for o in orbits_thick])
vy_f = np.array([safe_val(o.vy(ts[-1]), vo_val) for o in orbits_thick])
vz_f = np.array([safe_val(o.vz(ts[-1]), vo_val) for o in orbits_thick])

data = np.column_stack([x_i, y_i, z_i, vx_i, vy_i, vz_i, x_f, y_f, z_f, vx_f, vy_f, vz_f])
np.savetxt("orbits_thick.txt", data, header=header, fmt="%.6f")

print(f"✅ Saved {N} thick disk orbits to orbits_thick.txt")

✅ Saved 1000 thick disk orbits to orbits_thick.txt


# Buldge

In [23]:
# --- Parameters for bulge ---
sigma_R, sigma_T, sigma_z = 90, 90, 90  # km/s
mean_vT = 0                             # almost no net rotation

x0, y0, z0 = x_bulge.value, y_bulge.value, z_bulge.value
R0 = np.sqrt(x0**2 + y0**2)
phi0 = np.arctan2(y0, x0)
N = len(x0)

vR0 = np.random.normal(0, sigma_R, N) * u.km/u.s
vT0 = np.random.normal(mean_vT, sigma_T, N) * u.km/u.s
vz0 = np.random.normal(0, sigma_z, N) * u.km/u.s

orbits_bulge = [
    Orbit([R0[i]*u.kpc, vR0[i], vT0[i], z0[i]*u.kpc, vz0[i], phi0[i]*u.rad],
          ro=ro, vo=vo)
    for i in range(N)
]

for o in orbits_bulge:
    o.integrate(ts, pot, method='leapfrog')

print(f"✅ Integrated {N} bulge orbits for {t_end}.")

# Helper functions to handle both unitful and unitless outputs
def safe_val(val, scale=1.0):
    """Return numeric value from a Galpy output (Quantity or float)."""
    if hasattr(val, "to"):
        return val.value
    else:
        return np.array(val) * scale

# Use ro (kpc) and vo (km/s) for scaling
ro_val = ro.value if hasattr(ro, "value") else ro
vo_val = vo.value if hasattr(vo, "value") else vo

x_i = np.array([safe_val(o.x(ts[0]), ro_val) for o in orbits_bulge])
y_i = np.array([safe_val(o.y(ts[0]), ro_val) for o in orbits_bulge])
z_i = np.array([safe_val(o.z(ts[0]), ro_val) for o in orbits_bulge])

vx_i = np.array([safe_val(o.vx(ts[0]), vo_val) for o in orbits_bulge])
vy_i = np.array([safe_val(o.vy(ts[0]), vo_val) for o in orbits_bulge])
vz_i = np.array([safe_val(o.vz(ts[0]), vo_val) for o in orbits_bulge])

x_f = np.array([safe_val(o.x(ts[-1]), ro_val) for o in orbits_bulge])
y_f = np.array([safe_val(o.y(ts[-1]), ro_val) for o in orbits_bulge])
z_f = np.array([safe_val(o.z(ts[-1]), ro_val) for o in orbits_bulge])

vx_f = np.array([safe_val(o.vx(ts[-1]), vo_val) for o in orbits_bulge])
vy_f = np.array([safe_val(o.vy(ts[-1]), vo_val) for o in orbits_bulge])
vz_f = np.array([safe_val(o.vz(ts[-1]), vo_val) for o in orbits_bulge])

data = np.column_stack([x_i, y_i, z_i, vx_i, vy_i, vz_i, x_f, y_f, z_f, vx_f, vy_f, vz_f])
np.savetxt("orbits_bulge.txt", data, header=header, fmt="%.6f")

print(f"✅ Saved {N} bulge orbits to orbits_bulge.txt")

✅ Integrated 1000 bulge orbits for 2.0 Gyr.
✅ Saved 1000 bulge orbits to orbits_bulge.txt


# Halo

In [24]:
# --- Parameters for halo ---
sigma_R, sigma_T, sigma_z = 120, 120, 120  # km/s
mean_vT = 0                                # no rotation

x0, y0, z0 = x_halo.value, y_halo.value, z_halo.value
R0 = np.sqrt(x0**2 + y0**2)
phi0 = np.arctan2(y0, x0)
N = len(x0)

vR0 = np.random.normal(0, sigma_R, N) * u.km/u.s
vT0 = np.random.normal(mean_vT, sigma_T, N) * u.km/u.s
vz0 = np.random.normal(0, sigma_z, N) * u.km/u.s

orbits_halo = [
    Orbit([R0[i]*u.kpc, vR0[i], vT0[i], z0[i]*u.kpc, vz0[i], phi0[i]*u.rad],
          ro=ro, vo=vo)
    for i in range(N)
]

for o in orbits_halo:
    o.integrate(ts, pot)

print(f"✅ Integrated {N} halo orbits for {t_end}.")

sigma_R, sigma_T, sigma_z = 120, 120, 120
mean_vT = 0

x0, y0, z0 = x_halo.value, y_halo.value, z_halo.value
R0 = np.sqrt(x0**2 + y0**2)
phi0 = np.arctan2(y0, x0)
N = len(x0)

vR0 = np.random.normal(0, sigma_R, N) * u.km/u.s
vT0 = np.random.normal(mean_vT, sigma_T, N) * u.km/u.s
vz0 = np.random.normal(0, sigma_z, N) * u.km/u.s

orbits_halo = [
    Orbit([R0[i]*u.kpc, vR0[i], vT0[i], z0[i]*u.kpc, vz0[i], phi0[i]*u.rad],
          ro=ro, vo=vo)
    for i in range(N)
]

for o in orbits_halo:
    o.integrate(ts, pot, method='leapfrog')

    # Helper functions to handle both unitful and unitless outputs
def safe_val(val, scale=1.0):
    """Return numeric value from a Galpy output (Quantity or float)."""
    if hasattr(val, "to"):
        return val.value
    else:
        return np.array(val) * scale

# Use ro (kpc) and vo (km/s) for scaling
ro_val = ro.value if hasattr(ro, "value") else ro
vo_val = vo.value if hasattr(vo, "value") else vo


x_i = np.array([safe_val(o.x(ts[0]), ro_val) for o in orbits_halo])
y_i = np.array([safe_val(o.y(ts[0]), ro_val) for o in orbits_halo])
z_i = np.array([safe_val(o.z(ts[0]), ro_val) for o in orbits_halo])

vx_i = np.array([safe_val(o.vx(ts[0]), vo_val) for o in orbits_halo])
vy_i = np.array([safe_val(o.vy(ts[0]), vo_val) for o in orbits_halo])
vz_i = np.array([safe_val(o.vz(ts[0]), vo_val) for o in orbits_halo])

x_f = np.array([safe_val(o.x(ts[-1]), ro_val) for o in orbits_halo])
y_f = np.array([safe_val(o.y(ts[-1]), ro_val) for o in orbits_halo])
z_f = np.array([safe_val(o.z(ts[-1]), ro_val) for o in orbits_halo])

vx_f = np.array([safe_val(o.vx(ts[-1]), vo_val) for o in orbits_halo])
vy_f = np.array([safe_val(o.vy(ts[-1]), vo_val) for o in orbits_halo])
vz_f = np.array([safe_val(o.vz(ts[-1]), vo_val) for o in orbits_halo])

data = np.column_stack([x_i, y_i, z_i, vx_i, vy_i, vz_i, x_f, y_f, z_f, vx_f, vy_f, vz_f])
np.savetxt("orbits_halo.txt", data, header=header, fmt="%.6f")

print(f"✅ Saved {N} halo orbits to orbits_halo.txt")

✅ Integrated 641 halo orbits for 2.0 Gyr.
✅ Saved 641 halo orbits to orbits_halo.txt
